In [1]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.neural_network import MLPClassifier

In [2]:
# Importamos los datos

X=np.load("./datos_mfcc_pca99_X.npy")
y=np.load("./datos_mfcc_pca99_y.npy")

In [3]:
#Reducimos el conjunto de entrenamiento al 50% usando un método no supervisado. En este caso, un C-means con el que pretendemos
# quedarnos con los centros de los clusters que aparezcan, que serán los puntos más representativos.

c = 6             # número de clases
nc = 120          # muestras por clase
n= 720           # número de muestras totales

X_reduc=[]
y_reduc=np.int16(np.kron(np.arange(c),np.ones(nc//2)))

for i in range(0,n,nc):
    k_medoids=KMedoids(n_clusters=nc//2)
    k_medoids.fit(X[i:(i+nc)])
    centros=k_medoids.cluster_centers_
    X_reduc.append(centros)

In [4]:
print(X_reduc[0].shape)
# Reshapeamos las dimensiones de X_reduc para tenerlo todo en una matriz dónde cada fila será una muestra y cada columna una 
# característica. La matriz estará ordenada por clases. 
# Para reshapear, antes habrá que convertir a array.


X_reduc=np.array(X_reduc).reshape((360,464))
print(X_reduc.shape)

(60, 464)
(360, 464)


Una vez tenemos el conjunto reducido al 50%, podemos hacer la división en entrenamiento y test.

In [5]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [7]:
# Ahora podemos aplicar un algoritmo de tipo red neuronal a nuestros datos. Empezaremos entrenándolo con los parámetros por 
# defecto.

clf_MLP = MLPClassifier(random_state=0)     # Definimos un modelo de clasificación de una red neuronal, con todos los parámetros 
                                        # por defecto.
clf_MLP.fit(Xtr, ytr)       # Ajustamos la red con nuestros datos.
print(clf_MLP)
print('OA train %0.2f' % clf_MLP.score(Xtr, ytr))  # el método score calcula el error, en este caso en entrenamiento.
print('OA test %0.2f' % clf_MLP.score(Xts, yts))    # igual pero en test.

#La frontera de decisión sería la línea blanca, justo dónde ocurre el cambio de color.



MLPClassifier(random_state=0)
OA train 1.00
OA test 0.40


Parece un claro caso de sobre entrenamiento. Puede deberse a que los parámetros de la red neuronal no son los adecuados. Echamos un vistazo a los parámetros.

In [8]:
clf_MLP.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 0,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

En efecto, vemos que uno de los parámetros son las capas ocultas, y está puesto por defecto a 100 capas. Esto evidentemente es demasiado para nuestro problema. En principio con un par de capas ocultas debería de bastar para resolverlo.

Sin embargo, vamos a probar con otros parámetros para ver si mejora algo la cosa.

In [11]:
# Entrenamiento MLP (parámetros fijados)
#probamos otras opciones del clasificador buscando que el ajuste sea mejor.

clf_MLP2 = MLPClassifier(solver='lbfgs', alpha=1e-0, hidden_layer_sizes=(14, 5), random_state=0)  
clf_MLP2.fit(Xtr, ytr)

print(clf_MLP2)
print('OA train %0.2f' % clf_MLP2.score(Xtr, ytr)) 
print('OA test %0.2f' % clf_MLP2.score(Xts, yts))


MLPClassifier(alpha=1.0, hidden_layer_sizes=(14, 5), random_state=0)
OA train 0.62
OA test 0.18


C:\Users\Master\anaconda3\envs\MCD_21_22\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Obsevamos que no es un buen ajuste, pero al menos no tenemos un caso de overfitting. Al modificar los parámetros cambia la manera de entrenar el clasificador. 

Lo ideal es hacer una búsqueda de los parámetros ideales por cross-validation.

In [22]:
alphas = np.logspace(-3, 3, 4)   # 5 = número de combinaciones. Nos interesa mantenerlo pequeño.
neurons = [[i,j] for i in range(5,12,3) for j in range(2,6,2)]
tuned_parameters = {'solver': ['lbfgs'], 'alpha': alphas, 
                    'hidden_layer_sizes': neurons,'random_state': [0]}

#GridSearchCV nos permite hacer cross validation de forma automatica
# Usaremos el cross validation para optimizar los pesos. Haremos varios clasificadores de redes neuronales, y nos quedaremos con 
# el que nos dé los mejores pesos.

clf_MLP3 = GridSearchCV(MLPClassifier(solver='lbfgs'), tuned_parameters, cv=5,n_jobs=-1,verbose=2)
clf_MLP3.fit(Xtr,ytr)  # Con este fit, estamos ajustando todas las combinaciones obtenidas del cross validation.
clf_MLP3=clf_MLP3.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\Master\anaconda3\envs\MCD_21_22\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [23]:
#classifier.coefs_ contains the weight matrices that constitute the model parameters:
print([coef.shape for coef in clf_MLP3.coefs_])  

print(clf_MLP3)
print('OA train %0.2f' % clf_MLP3.score(Xtr, ytr)) 
print('OA test %0.2f' % clf_MLP3.score(Xts, yts))


[(464, 11), (11, 4), (4, 6)]
MLPClassifier(alpha=10.0, hidden_layer_sizes=[11, 4], random_state=0,
              solver='lbfgs')
OA train 0.99
OA test 0.50


Parece un caso de overfitting de nuevo. Los parámetros óptimos no parecen ser buenos tampoco. 

Quizás lo que ocurre es que tenemos pocos datos de entrenamiento. Las redes neuronales necesitan un conjunto de entrenamiento grande para tener un buen rendimiento. Por lo tanto, quizás este método no es el adecuado para resolver nuestro problema.

In [25]:
preds_test = clf_MLP3.predict(Xts)
CM_MLP=metrics.confusion_matrix(yts,preds_test)
print(CM_MLP)

[[ 7  2  1  1  3  3]
 [ 0 15  0  0  2  3]
 [ 1  5  4  7  2  1]
 [ 0  3  2 16  1  0]
 [ 3  4  0  2  8  1]
 [ 3  2  0  0  2  4]]
